In [68]:
from nltk.tokenize import TreebankWordTokenizer
from pymystem3 import Mystem
from nltk.tokenize import sent_tokenize
import copy
import pandas as pd
from collections import Counter
import nltk

#fp = open('Pelevin_Fuji_raw.txt')
#data = fp.read()
#sent_split = sent_tokenize(data)

In [69]:
tokenizer = TreebankWordTokenizer()

In [70]:
doc_tokens = []
for line in sent_split:    
    doc_tokens += [sorted(tokenizer.tokenize(line.lower()))]

In [71]:
puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
            ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
stopwords = nltk.corpus.stopwords.words('russian')
stopwords += set(('это', 'сказал', 'сказала', 'очень', 'просто', 'такое', 'что-то', 'тебе',
                 'вообще','сразу','несколько','почему','именно','самое','спросил','сказать','ответила'))
all_doc_tokens = sum(doc_tokens, [])

word_list = [x for x in all_doc_tokens 
             if x not in puncs 
             if x not in stopwords]

lexicon = sorted(set(word_list))
count_words = Counter(word_list)
print('Объем лексикона (словоформы) без стопслов:', len(lexicon))
print('Количество токенов без стопслов и знаков препинания:', len(word_list))
#count_words.most_common(30)

Объем лексикона (словоформы) без стопслов: 19652
Количество токенов без стопслов и знаков препинания: 47104


In [72]:
fp = open('Pelevin_Fuji_raw.txt', 'r')
data = fp.read()

def clean_chapters(data):
    memo_lemmas_copy = []
    for chapter in data.split('\n\n'):
        memo_lemmas_copy += [copy.copy(chapter)]

    return memo_lemmas_copy

corpus_ch = clean_chapters(data)
#corpus_ch

In [73]:

fp = open('Pelevin_Fuji_raw.txt', 'r')
data = fp.read()
sent_split = sent_tokenize(data)
len(sent_split)

"""Splits text on sentences and saves them into a list without keeping chapters splitted. 
    Removes punctuation"""
def clean_sentences(text):
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                 ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    memo_lemmas_copy = []
    for sentence in sent_split:
        memo_lemmas_copy += [copy.copy(sentence)]
        
    memo_lemmas_copy = [(letter.replace('\n', ' ') for letter in sentence) #turn it off to split chapters
                       for sentence in memo_lemmas_copy]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]

    df = pd.DataFrame(memo_lemmas_copy, columns=['text'])
    return df

corpus = clean_sentences(data)
#corpus

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=corpus.text).toarray()

tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
#tfidf_docs.shape

In [75]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
index = ['sentence{}'.format(i) for i in range(len(corpus))]
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
#pca_topic_vectors.round(2).head(106)

In [76]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
#terms

In [77]:
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns = 8
#weights.head(10).round(3)

In [78]:
pd.options.display.max_columns = 12
deals = weights['душа тело женщина душа тяжесть'.split()].round(3)*100
#deals

In [79]:
#deals.T.sum()

In [80]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
#svd_topic_vectors.round(3).head(6)

In [45]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=corpus.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms

In [81]:
#corpus.loc[50].text

In [82]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
#ldia.components_.shape

In [83]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
#components.round(2).head(3)

In [84]:
#components.topic11.sort_values(ascending=False)[:60]